In [ ]:
import psutil
print(f"Memoria disponible: {psutil.virtual_memory().available / 1e9} GB")

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, SimpleRNN, GRU, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt


Memoria disponible: 12.35503104 GB


In [ ]:
# Cargar datos
# data = pd.read_csv("Datos_ConT.csv")
data = pd.read_csv("Datos_Con2HResampled.csv")
data = data.rename(columns={'DateTime_': 'Fecha'})
data['Fecha'] = pd.to_datetime(data['Fecha'])


In [ ]:
# Mover columna 'Magnitud'
magnitud_column = data['Magnitud']
data = data.drop('Magnitud', axis=1)
data.insert(0, 'Magnitud', magnitud_column)

In [ ]:
# Normalización
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Fecha']))

In [ ]:
# Crear secuencias
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :])
        y.append(data[i+seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 48
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

n_features = data.shape[1] - 1
l2_reg = l2(0.01)

In [ ]:
# Modelo LSTM
model_lstm = Sequential([
    Bidirectional(LSTM(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg), input_shape=(seq_length, n_features)),
    Dropout(0.3),
    Bidirectional(LSTM(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg)),
    Dropout(0.3),
    Bidirectional(LSTM(50, activation='tanh', kernel_regularizer=l2_reg)),
    Dropout(0.3),
    Dense(1)
])

# Modelos RNN y GRU
model_rnn = Sequential([
    SimpleRNN(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg, input_shape=(seq_length, n_features)),
    Dropout(0.3),
    SimpleRNN(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg),
    Dropout(0.3),
    SimpleRNN(50, activation='tanh', kernel_regularizer=l2_reg),
    Dropout(0.3),
    Dense(1)
])

model_gru = Sequential([
    GRU(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg, input_shape=(seq_length, n_features)),
    Dropout(0.3),
    GRU(50, return_sequences=True, activation='tanh', kernel_regularizer=l2_reg),
    Dropout(0.3),
    GRU(50, activation='tanh', kernel_regularizer=l2_reg),
    Dropout(0.3),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Entrenar modelos
for model, name in [(model_lstm, "LSTM"), (model_rnn, "RNN"), (model_gru, "GRU")]:
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print(f"Entrenando modelo {name}...")
    model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_test, y_test), verbose=1)

Entrenando modelo LSTM...
Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 54s 347ms/step - loss: 3.3717 - mae: 0.0107 - val_loss: 0.3265 - val_mae: 0.1600
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 79s 325ms/step - loss: 0.1349 - mae: 0.0092 - val_loss: 0.1248 - val_mae: 0.2015
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 333ms/step - loss: 0.0034 - mae: 0.0084 - val_loss: 0.1194 - val_mae: 0.2024
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 42s 341ms/step - loss: 0.0015 - mae: 0.0089 - val_loss: 0.1189 - val_mae: 0.2027
Epoch 5/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 343ms/step - loss: 0.0017 - mae: 0.0098 - val_loss: 0.1188 - val_mae: 0.2028
Epoch 6/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 39s 324ms/step - loss: 0.0016 - mae: 0.0100 - val_loss: 0.1203 - val_mae: 0.2017
Epoch 7/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 38s 322ms/step - loss: 0.0015 - mae: 0.0085 - val_loss: 0.1198 - val_mae: 0.2021
Epoch 8/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 42s 335ms/step - loss: 0.0017 - mae: 0.0096 - val_loss: 0.1202 - val_mae: 0.2018
Epoch 

In [ ]:
# Evaluar modelos
for model, name in [(model_lstm, "LSTM"), (model_rnn, "RNN"), (model_gru, "GRU")]:
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    print(f"Resultados para {name}:")
    print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r2:.4f}\n")

119/119 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step
Resultados para LSTM:
MAE: 0.2017, MSE: 0.1203, RMSE: 0.3469, R^2: -0.4654

119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Resultados para RNN:
MAE: 0.2020, MSE: 0.1199, RMSE: 0.3463, R^2: -0.4607

119/119 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step
Resultados para GRU:
MAE: 0.2014, MSE: 0.1207, RMSE: 0.3475, R^2: -0.4703



In [ ]:
# Función para predicciones futuras
def predict_for_dates(model, df, scaler, start_date, days_to_predict):
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    start_date = pd.to_datetime(start_date)
    input_data = df[df['Fecha'] >= start_date].drop(columns=['Fecha']).values
    input_scaled = scaler.transform(input_data)[-seq_length:]

    predictions = []
    current_input = np.array([input_scaled])

    for _ in range(days_to_predict):
        pred = model.predict(current_input)[0][0]
        predictions.append(pred)
        new_input = np.hstack([[pred], np.zeros(n_features - 1)])
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1, :] = new_input

    return predictions

# Predicción futura
start_date = data['Fecha'].max()
days_to_predict = 12
future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="2H")

future_predictions_lstm = predict_for_dates(model_lstm, data, scaler, start_date, days_to_predict)
future_predictions_rnn = predict_for_dates(model_rnn, data, scaler, start_date, days_to_predict)
future_predictions_gru = predict_for_dates(model_gru, data, scaler, start_date, days_to_predict)

predictions_df = pd.DataFrame({
    "Fecha": future_dates,
    "LSTM": future_predictions_lstm,
    "RNN": future_predictions_rnn,
    "GRU": future_predictions_gru
})

print(predictions_df)

<ipython-input-9-a344f87c406f>:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(start=start_date, periods=days_to_predict, freq="2H")
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 875ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
                 Fecha      LSTM       RNN       GRU
0  2022-02-10 06:00:00  0.004642  0.003338  0.003991
1  2022-02-10 08:00:00  0.004642  0.003338  0.003991
2  2022-02-10 10:00:00  0.004642  0.003338  0.003991
3  2022-02-10 12:00:00  0.004642  0.003338  0.003991
4  2022-02-10 14:00:00  0.004642  0.003338  0.003991
5  2022-02-10 16:00:00  0.004642  0.003338  0.003991
6  2022-02-10 18:00:00  0.004642  0.003338  0.003991
7  2022-02-10 20:00:00  0.004642  0.003338  0.003991
8  2022-02-10 22:00:00  0.004642  0.003338  0.003991
9  2022-02-11